我们可以把RAG当作Agent可以调用的一个工具。

先配置对话模型和嵌入模型。模型的构建可以参考wow-rag课程的第二课（https://github.com/datawhalechina/wow-rag/tree/main/tutorials），里面介绍了非常多配置对话模型和嵌入模型的方式。这里采用了本地Ollama的对话模型和嵌入模型。各种配置方式都可以，只要能有个能用的llm和embedding就行。

如果运行还算顺利，可以顺便给wow-rag和wow-agent项目都点个小星星吗？谢谢！！！

> ⚠️ 为了简化操作，依旧使用智谱 AI 的 glm-4-flash 和 embedding-3 ，请自行申请：https://www.bigmodel.cn/invite?icode=tzUaambCl5Rc0B964McaGGczbXFgPRGIalpycrEwJ28%3D

In [24]:
# 配置chat模型
from llama_index.llms.zhipuai import ZhipuAI

llm = ZhipuAI(model="glm-4-flash", api_key="YOUR API KEY")
llm = ZhipuAI(model="glm-4-flash", api_key="2ce2c9417f83dc0f582c00498b2bbd8b.374mpCJDIXF5ksmq")

In [26]:
# 配置Embedding模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding

embedding = ZhipuAIEmbedding(model="embedding-3", api_key="YOUR API KEY")
embedding = ZhipuAIEmbedding(model="embedding-3", api_key="2ce2c9417f83dc0f582c00498b2bbd8b.374mpCJDIXF5ksmq")

上边这个llm和embedding有很多方法可以构建。详情参见wow-rag的第二课。

In [4]:
# 测试对话模型
response = llm.complete("你是谁？")
print(response)

我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


In [5]:
# 测试嵌入模型
emb = embedding.get_text_embedding("你是谁？")
len(emb), type(emb)

(1024, list)

输出 (1024, list)

说明配置成功。



然后构建索引

In [11]:
pip install faiss-cpu -i https://mirrors.aliyun.com/pypi/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
     --- ------------------------------------ 1.0/13.7 MB 5.6 MB/s eta 0:00:03
     ------ --------------------------------- 2.4/13.7 MB 5.6 MB/s eta 0:00:03
     ---------- ----------------------------- 3.7/13.7 MB 5.7 MB/s eta 0:00:02
     ------------- -------------------------- 4.7/13.7 MB 5.6 MB/s eta 0:00:02
     ----------------- ---------------------- 6.0/13.7 MB 5.6 MB/s eta 0:00:02
     --------------------- ------------------ 7.3/13.7 MB 5.6 MB/s eta 0:00:02
     ------------------------- -------------- 8.7/13.7 MB 5.6 MB/s eta 0:00:01
     ----------------------------- ---------- 10.0/13.7 MB 5.6 MB/s eta 0:00:01
     -------------------------------- ------- 11.0/13.7 MB 5.6 MB/s eta 0:00:01
     ------------------------------------ --- 12.3/13.7 MB 5.6 MB/s eta 0:00:0

In [39]:
# 从指定文件读取，输入为List
from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['../docs/问答手册.txt', '../docs/商标注册.txt']).load_data()

# 构建节点
from llama_index.core.node_parser import SentenceSplitter
transformations = [SentenceSplitter(chunk_size = 512)]

from llama_index.core.ingestion.pipeline import run_transformations
nodes = run_transformations(documents, transformations=transformations)

# 构建索引
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
from llama_index.core import StorageContext, VectorStoreIndex

emb = embedding.get_text_embedding("你好呀呀")
vector_store = FaissVectorStore(faiss_index=faiss.IndexFlatL2(len(emb)))
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes = nodes,
    storage_context=storage_context,
    embed_model = embedding,
)

然后构建问答引擎

In [42]:
# 构建检索器
from llama_index.core.retrievers import VectorIndexRetriever
# 想要自定义参数，可以构造参数字典
kwargs = {'similarity_top_k': 5, 'index': index, 'dimensions': len(emb)} # 必要参数
retriever = VectorIndexRetriever(**kwargs)

# 构建合成器
from llama_index.core.response_synthesizers  import get_response_synthesizer
response_synthesizer = get_response_synthesizer(llm=llm, streaming=True)

# 构建问答引擎
from llama_index.core.query_engine import RetrieverQueryEngine
engine = RetrieverQueryEngine(
      retriever=retriever,
      response_synthesizer=response_synthesizer,
        )

用RAG回答一下试试效果：

In [29]:
# 提问
question = "What are the applications of Agent AI systems ?"
response = engine.query(question)
for text in response.response_gen:
    print(text, end="")

Agent AI systems can be applied in interactive AI, content generation for bots and AI agents, productivity applications such as content replay, paraphrasing, action prediction, and synthesis of 3D or 2D scenarios. These systems also have potential applications in health topics, including ethical deployment in sensitive domains like healthcare. They can transform the gaming industry by shifting developer focus and refine agent learning processes, as well as redefine manufacturing roles with adaptive robotic systems.

理想输出

```
Agent AI systems have a variety of applications, which include:

1. Interactive AI: Enhancing user interactions and providing personalized experiences.
2. Content Generation: Assisting in the creation of content for bots and AI agents, which can be used in various applications such as customer service or storytelling.
3. Productivity: Improving productivity in applications by enabling tasks like replaying events, paraphrasing information, predicting actions, and synthesizing scenarios (both 3D and 2D).
4. Healthcare: Ethical deployment in sensitive domains like healthcare, which could potentially improve diagnoses and patient care while also addressing health disparities.
5. Gaming Industry: Transforming the role of developers by shifting focus from scripting non-player characters to refining agent learning processes.
6. Robotics and Manufacturing: Redefining manufacturing roles and requiring new skill sets, rather than replacing human workers, as adaptive robotic systems are developed.
7. Simulation: Learning collaboration policies within simulated environments, which can be applied to the real world with careful consideration and safety measures.
```


我们可以把这个RAG当作一个工具给Agent调用，让它去思考。
先来配置问答工具

In [43]:
# 配置查询工具
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="RAG工具",
            description=(
                "用于在原文中检索相关信息"
            ),
        ),
    ),
]

创建ReAct Agent

In [37]:
# 创建ReAct Agent
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

调用Agent

In [44]:
# 让Agent完成任务
response = agent.chat("请问商标注册需要提供哪些文件？")
# response = agent.chat("What are the applications of Agent AI systems ?")
print(response)

> Running step 96208f24-d453-47c8-8085-b19d6d148559. Step input: 请问商标注册需要提供哪些文件？
Thought: The user is asking about the documents required for trademark registration. I need to use a tool to help me answer this question.
Action: RAG工具
Action Input: {'input': '商标注册需要提供哪些文件？'}
Observation: 商标注册需要提供的文件通常包括但不限于以下内容：

1. 商标注册申请书：填写商标注册申请的相关信息，包括申请人信息、商标图样、商品或服务类别等。
2. 申请人身份证明文件：如个人身份证、企业营业执照等。
3. 商标图样：清晰、准确的商标图样，可以是黑白或彩色，但必须是清晰可辨的。
4. 商品或服务列表：详细列出商标所适用的商品或服务类别，需按照《商标注册用商品和服务国际分类》进行分类。
5. 商标代理委托书：如委托代理人办理商标注册事宜，需提供委托书，并附上代理人身份证明文件。
6. 其他相关文件：根据具体情况，可能还需要提供其他文件，如变更申请、续展申请等。

请注意，具体所需文件可能因国家或地区而异，建议咨询当地商标注册机构或专业律师。
> Running step 990d9d6e-d285-48dc-aa15-ad35a7035513. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 商标注册需要提供的文件通常包括但不限于以下内容：商标注册申请书、申请人身份证明文件、商标图样、商品或服务列表、商标代理委托书以及可能的其他相关文件。具体所需文件可能因国家或地区而异，建议咨询当地商标注册机构或专业律师。
商标注册需要提供的文件通常包括但不限于以下内容：商标注册申请书、申请人身份证明文件、商标图样、商品或服务列表、商标代理委托书以及可能的其他相关文件。具体所需文件可能因国家或地区而异，建议咨询当地商标注册机构或专业律

理想输出:
```
Thought: 我需要使用工具来获取关于商标注册所需文件的信息。
Action: RAG
Action Input: {'input': '商标注册需要提供哪些文件'}
Observation: 商标注册通常需要以下文件：

1. **企业**：申请人为企业的，则需提供：
   - 营业执照复印件
   - 授权委托书（如果由代理人提交）
   - 商标图案电子档
   - 具体商品或服务的名称

2. **国内自然人**：以个人名义申请时，需要提供：
   - 个体工商户档案（如有营业执照）
   - 自然人身份证复印件
   - 授权委托书（如果由代理人提交）
   - 商标图案电子档
   - 具体商品或服务的名称

3. **国外自然人**：申请商标时，通常需要：
   - 护照复印件（作为身份证明文件）
   - 授权委托书（如果由代理人提交）
   - 商标图案电子档
   - 具体商品或服务的名称

请注意，具体要求可能会因国家和地区政策的不同而有所变化。在实际申请前，请咨询当地的知识产权局或专业代理机构以获取最准确的信息。
Thought: 我可以使用这些信息来回答问题。
Answer: 商标注册通常需要以下文件：

1. **企业**：营业执照复印件、授权委托书（如果由代理人提交）、商标图案电子档以及具体商品或服务的名称。
2. **国内自然人**：个体工商户档案（如有）、自然人身份证复印件、授权委托书（如果由代理人提交）、商标图案电子档和具体商品或服务的名称。
3. **国外自然人**：护照复印件作为身份证明文件、授权委托书（如果由代理人提交）、商标图案电子档以及具体商品或服务的名称。

请注意，具体的申请要求可能会因国家和地区政策的不同而有所变化。在实际申请前，请咨询当地的知识产权局或专业代理机构以获取最准确的信息。
商标注册通常需要以下文件：

1. **企业**：营业执照复印件、授权委托书（如果由代理人提交）、商标图案电子档以及具体商品或服务的名称。
2. **国内自然人**：个体工商户档案（如有）、自然人身份证复印件、授权委托书（如果由代理人提交）、商标图案电子档和具体商品或服务的名称。
3. **国外自然人**：护照复印件作为身份证明文件、授权委托书（如果由代理人提交）、商标图案电子档以及具体商品或服务的名称。

请注意，具体的申请要求可能会因国家和地区政策的不同而有所变化。在实际申请前，请咨询当地的知识产权局或专业代理机构以获取最准确的信息。
````

看起来这个回答比单纯使用RAG的效果好很多。
⚠️效果依赖于大模型的能力，如果大模型能力不足，那么效果可能还不如RAG。